In [1]:
import gc
import pandas as pd
import numpy as np
import os
import xgboost as xgb
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier
from sklearn.externals import joblib
from imblearn.over_sampling import SMOTE 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve 

In [2]:
def score(y,pred): 
    fpr, tpr, thresholds = roc_curve(y, pred, pos_label=1) 
    score=0.4*tpr[np.where(fpr>=0.001)[0][0]]+0.3*tpr[np.where(fpr>=0.005)[0][0]]+0.3*tpr[np.where(fpr>=0.01)[0][0]] 
    return score 

In [105]:
#################### lightGBM单模型 ###########################
def LGBModel(X_train, y_train, X_val, y_val, random_state, cols, col_int):
    clf = lgb.LGBMClassifier(  
                                boosting_type='gbdt',
                                num_leaves=55,
                                #max_depth=-1,
                                n_estimators=300,
                                #max_bin=50,
                                objective='binary',
                                #min_split_gain=0.3,
                                learning_rate=0.05,
                                feature_fraction=0.9,
                                bagging_fraction=0.8,
                                random_state=random_state,
                                #reg_alpha=0.7,
                                #reg_lambda=0.4,
                                bagging_freq=5,
                                verbose=0,
                                n_jobs=-1     
    )
    clf.fit(
                X_train, y_train, 
                eval_set=[(X_val, y_val), (X_train, y_train)], 
                eval_metric='auc', 
                early_stopping_rounds=10,
                feature_name=cols,
                categorical_feature=col_int
    )
    return clf

In [35]:
def sampling(Dataframe, frac, random_state):
    Dataframe_sampled = Dataframe.sample(
        n=None, 
        frac=frac, 
        replace=False, 
        weights=None, 
        random_state=random_state, 
        axis=0
    )
    return Dataframe_sampled

In [54]:
def OnehotEncoder(Dataframe, col_int):
    onehot = []
    for col in col_int:
        dummies = pd.get_dummies(Dataframe[col], prefix=col)
        onehot.append(dummies)

    for i in onehot:
        data = pd.concat([Dataframe, i], axis=1)

    Dataframe.drop(col_int, axis=1, inplace=True)
    return Dataframe

In [106]:
####################训练集#######################
DATA_PATH = "datasets"
csv_path = os.path.join(DATA_PATH, "atec_anti_fraud_train.csv")
data = pd.read_csv(csv_path)

In [113]:
####################训练集#######################
pd.set_option('display.max_columns',2000)
data

,id,label,date,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118,f119,f120,f121,f122,f123,f124,f125,f126,f127,f128,f129,f130,f131,f132,f133,f134,f135,f136,f137,f138,f139,f140,f141,f142,f143,f144,f145,f146,f147,f148,f149,f150,f151,f152,f153,f154,f155,f156,f157,f158,f159,f160,f161,f162,f163,f164,f165,f166,f167,f168,f169,f170,f171,f172,f173,f174,f175,f176,f177,f178,f179,f180,f181,f182,f183,f184,f185,f186,f187,f188,f189,f190,f191,f192,f193,f194,f195,f196,f197,f198,f199,f200,f201,f202,f203,f204,f205,f206,f207,f208,f209,f210,f211,f212,f213,f214,f215,f216,f217,f218,f219,f220,f221,f222,f223,f224,f225,f226,f227,f228,f229,f230,f231,f232,f233,f234,f235,f236,f237,f238,f239,f240,f241,f242,f243,f244,f245,f246,f247,f248,f249,f250,f251,f252,f253,f254,f255,f256,f257,f258,f259,f260,f261,f262,f263,f264,f265,f266,f267,f268,f269,f270,f271,f272,f273,f274,f275,f276,f277,f278,f279,f280,f281,f282,f283,f284,f285,f286,f287,f288,f289,f290,f291,f292,f293,f294,f295,f296,f297
0,f10eb20f31cf7063ee8bdbd1272214e4d7e0193c8dbce4...,1,20171103,0,0,0,0,100807.0,0,5,1,1,1,1,2,0,2,0,2,2,2,1,31.0,61.0,142.0,245.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,3.0,96.0,38.0,301.0,302.0,302.0,302.0,301.0,302.0,40.0,79.0,40.0,79.0,2.0,3.0,4.0,3.0,41.0,80.0,96.5,721.1,3252.0,154.5,394.5,2.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,4.0,8.0,15.0,30.0,4.0,14.0,34.0,70.0,142.0,3.0,15.0,41.0,79.0,191.0,192.0,190.0,190.0,190.0,190.0,191.0,192.0,191.0,190.0,190.0,190.0,32.0,33.0,25.0,25.0,24.0,25.0,31.0,79.0,42.0,42.0,42.0,68.0,32.0,32.0,28.0,27.0,28.0,27.0,32.0,235.0,235.0,126.0,126.0,127.0,126.0,235.0,192.0,191.0,190.0,189.0,189.0,301.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,3.0,3.0,4.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,4.0,3.0,4.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,3.0,3.0,3.0,14.0,41.0,80.0,167.0,2.0,2.0,4.0,3.0,3.0,4.0,6.0,6.0,2.0,2.0,4.0,3.0,2.0,2.0,3.0,4.0,3.0,3.0,4.0,1.0,3.0,3.0,4.0,2.0,2.0,2.0,2.0,8.0,2.0,2.0,3.0,3.0,3.0,2.0,5.0,5.0,10.0,16.0,2.0,2.0,4.0,5.0,7.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,27.0,297.0,302.0,384.0,770.0,20.0,99.0,102.0,124.0,140.0,301.0,312.0,328.0,85.0,302.0,201.0,203.0,203.0,61.0,201.0
1,d861929b67938d06538b910b9f6b85f5eb62b6ad7361ba...,1,20170917,0,1,1,1,100805.0,1,5,2,1,1,0,2,1,2,2,1,1,2,0,14.0,13.0,48.0,48.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,2.0,4.0,4.0,5.0,1.0,1.0,4.0,3.0,3.0,79.0,12.0,302.0,32.0,301.0,302.0,302.0,302.0,23.0,26.0,23.0,26.0,2.0,2.0,3.0,4.0,23.0,26.0,5.0,717.4,733.3,231.8,472.8,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0,4.0,8.0,11.0,12.0,4.0,12.0,23.0,26.0,27.0,5.0,12.0,24.0,27.0,122.0,121.0,16.0,121.0,121.0,122.0,121.0,122.0,121.0,16.0,121.0,122.0,32.0,31.0,13.0,27.0,26.0,27.0,32.0,79.0,40.0,39.0,39.0,63.0,31.0,31.0,15.0,28.0,29.0,29.0,31.0,231.0,232.0,17.0,99.0,99.0,98.0,232.0,121.0,122.0,17.0,121.0,121.0,302.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,4.0,5.0,12.0,24.0,26.0,27.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,3.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1

In [108]:
cols = list(data.columns)[3:]
print(cols)

['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99', 'f100', 'f101', 'f102', 'f103', 'f104', 'f105', 'f106', 'f107', 'f108', 'f109', 'f110', 'f111', 'f112', 'f113', 'f114', 'f115', 'f116', 'f117', 'f118', 'f119', 'f120', 'f121', 'f122', 'f123', 'f124', 'f125', 'f126', 'f127', 'f128', 'f129', 'f130', 'f131', 'f132', 'f133', 'f134', 'f135', 'f136', 'f137', 'f138', 'f1

In [109]:
col_int = []
col_float = []
for col in cols:
    #print(data[col].dtype)
    if data[col].dtype == "int64":
        col_int.append(col)
    if data[col].dtype == "float64":
        col_float.append(col)
print(col_int)
print(col_float)

['f1', 'f2', 'f3', 'f4', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19']
['f5', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99', 'f100', 'f101', 'f102', 'f103', 'f104', 'f105', 'f106', 'f107', 'f108', 'f109', 'f110', 'f111', 'f112', 'f113', 'f114', 'f115', 'f116', 'f117', 'f118', 'f119', 'f120', 'f121', 'f122', 'f123', 'f124', 'f125', 'f126', 'f127', 'f128', 'f129', 'f130', 'f131', 'f132', 'f133', 'f134', 'f135', 'f136', 'f137', 'f138', 'f

In [7]:
##############训练集onehot编码#####################
data = OnehotEncoder(data, col_int)

In [110]:
##################缺失值填充########################
data.fillna(0, inplace=True)

In [111]:
data.loc[data['label'] == -1, 'label'] = 1
data.loc[data['label'] == 1, 'label'] = 2
data.loc[data['label'] == 0, 'label'] = 1
data.loc[data['label'] == 2, 'label'] = 0
data['label'].value_counts()

1    977884
0     16847
Name: label, dtype: int64

In [119]:
###################数据集划分（线下）########################
off_train = data[data['date'] <= 20171020]
off_val = data[(data['date'] <= 20171027) & (data['date'] > 20171020)]
off_test = data[data['date'] > 20171027]
print(off_train.shape)
print(off_val.shape)
print(off_test.shape)

(733683, 300)
(112915, 300)
(148133, 300)


In [12]:
# del data
# gc.collect()

2772

In [120]:
off_test['label'].value_counts()

1    145538
0      2595
Name: label, dtype: int64

In [80]:

# off_train1 = sampling(off_train[off_train['label'] == 1], 0.3, 2018)
# off_train1 = pd.concat([off_train1, off_train[off_train['label'] == 0]], axis=0)

# off_train2 = sampling(off_train[off_train['label'] == 1], 0.3, 2019)
# off_train2 = pd.concat([off_train2, off_train[off_train['label'] == 0]], axis=0)

# off_train3 = sampling(off_train[off_train['label'] == 1], 0.3, 2020)
# off_train3 = pd.concat([off_train3, off_train[off_train['label'] == 0]], axis=0)
# print(off_train1.shape)
# print(off_train2.shape)
# print(off_train3.shape)

(253808, 343)
(253808, 343)
(253808, 343)


In [81]:
# print(off_train1['label'].value_counts())
# print(off_train2['label'].value_counts())
# print(off_train3['label'].value_counts())

1    240100
0     13708
Name: label, dtype: int64
1    240100
0     13708
Name: label, dtype: int64
1    240100
0     13708
Name: label, dtype: int64


In [121]:
###################数据集划分（线下）########################
X_train = off_train.loc[:,'f1':].as_matrix()
y_train = off_train.loc[:,'label'].as_matrix()
print(X_train.shape, y_train.shape)

# X_train1 = off_train1.loc[:,'f5':].as_matrix()
# y_train1 = off_train1.loc[:,'label'].as_matrix()
# print(X_train1.shape, y_train1.shape)

# X_train2 = off_train2.loc[:,'f5':].as_matrix()
# y_train2 = off_train2.loc[:,'label'].as_matrix()
# print(X_train2.shape, y_train2.shape)

# X_train3 = off_train3.loc[:,'f5':].as_matrix()
# y_train3 = off_train3.loc[:,'label'].as_matrix()
# print(X_train3.shape, y_train3.shape)

X_val = off_val.loc[:,'f1':].as_matrix()
y_val = off_val.loc[:,'label'].as_matrix()
print(X_val.shape, y_val.shape)

X_test = off_test.loc[:,'f1':].as_matrix()
y_test = off_test.loc[:,'label'].as_matrix()
print(X_test.shape, y_test.shape)

(733683, 297) (733683,)
(112915, 297) (112915,)
(148133, 297) (148133,)


In [123]:
###################数据集划分（线下）########################
for i in range(len(y_test)):
    if y_test[i] == 1:
        y_test[i] = 0
    else:
        y_test[i] = 1

In [124]:
del off_train, off_val, off_test
gc.collect()

1090

In [125]:
clf = LGBModel(X_train, y_train, X_val, y_val, 2018, cols, col_int)

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:1038: UserWarning: categorical_feature in Dataset is overrided. New categorical_feature is ['f1', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f2', 'f3', 'f4', 'f6', 'f7', 'f8', 'f9']
  warnings.warn('categorical_feature in Dataset is overrided. New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[1]	valid_0's auc: 0.952294	valid_1's auc: 0.938752
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's auc: 0.958399	valid_1's auc: 0.948612
[3]	valid_0's auc: 0.95851	valid_1's auc: 0.949426
[4]	valid_0's auc: 0.959716	valid_1's auc: 0.950535
[5]	valid_0's auc: 0.960615	valid_1's auc: 0.952861
[6]	valid_0's auc: 0.960937	valid_1's auc: 0.954308
[7]	valid_0's auc: 0.961104	valid_1's auc: 0.954457
[8]	valid_0's auc: 0.961303	valid_1's auc: 0.9545
[9]	valid_0's auc: 0.962273	valid_1's auc: 0.955653
[10]	valid_0's auc: 0.962336	valid_1's auc: 0.955738
[11]	valid_0's auc: 0.964312	valid_1's auc: 0.958609
[12]	valid_0's auc: 0.964233	valid_1's auc: 0.958631
[13]	valid_0's auc: 0.964568	valid_1's auc: 0.958697
[14]	valid_0's auc: 0.964863	valid_1's auc: 0.95911
[15]	valid_0's auc: 0.964997	valid_1's auc: 0.959278
[16]	valid_0's auc: 0.965105	valid_1's auc: 0.959796
[17]	valid_0's auc: 0.965373	valid_1's auc: 0.959742
[18]	valid_0's auc: 0.965569	valid_1's auc: 0.9599

In [126]:
y_pred = clf.predict_proba(X_test)[:,0]
print(score(y_test, y_pred))

0.4225818882466281


In [86]:
clf2 = LGBModel(X_train2, y_train2, X_val, y_val, 2019)

[1]	valid_0's auc: 0.947073	valid_1's auc: 0.946701
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's auc: 0.958153	valid_1's auc: 0.954411
[3]	valid_0's auc: 0.959484	valid_1's auc: 0.955746
[4]	valid_0's auc: 0.960127	valid_1's auc: 0.95674
[5]	valid_0's auc: 0.96397	valid_1's auc: 0.959861
[6]	valid_0's auc: 0.964923	valid_1's auc: 0.960589
[7]	valid_0's auc: 0.965787	valid_1's auc: 0.966563
[8]	valid_0's auc: 0.966993	valid_1's auc: 0.967005
[9]	valid_0's auc: 0.966375	valid_1's auc: 0.967285
[10]	valid_0's auc: 0.966305	valid_1's auc: 0.967708
[11]	valid_0's auc: 0.965949	valid_1's auc: 0.967827
[12]	valid_0's auc: 0.965769	valid_1's auc: 0.968588
[13]	valid_0's auc: 0.966047	valid_1's auc: 0.969087
[14]	valid_0's auc: 0.966031	valid_1's auc: 0.969106
[15]	valid_0's auc: 0.966122	valid_1's auc: 0.969295
[16]	valid_0's auc: 0.965932	valid_1's auc: 0.969427
[17]	valid_0's auc: 0.965698	valid_1's auc: 0.969735
[18]	valid_0's auc: 0.965581	valid_1's auc: 0.96

In [87]:
y_pred2 = clf2.predict_proba(X_test)[:,0]
print(score(y_test, y_pred2))

0.373179190751445


In [88]:
clf3 = LGBModel(X_train3, y_train3, X_val, y_val, 2020)

[1]	valid_0's auc: 0.942836	valid_1's auc: 0.944245
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's auc: 0.953116	valid_1's auc: 0.957224
[3]	valid_0's auc: 0.960068	valid_1's auc: 0.962181
[4]	valid_0's auc: 0.964215	valid_1's auc: 0.964203
[5]	valid_0's auc: 0.964501	valid_1's auc: 0.964369
[6]	valid_0's auc: 0.964247	valid_1's auc: 0.965288
[7]	valid_0's auc: 0.964328	valid_1's auc: 0.965268
[8]	valid_0's auc: 0.964531	valid_1's auc: 0.965343
[9]	valid_0's auc: 0.964144	valid_1's auc: 0.966741
[10]	valid_0's auc: 0.964676	valid_1's auc: 0.967137
[11]	valid_0's auc: 0.964412	valid_1's auc: 0.968424
[12]	valid_0's auc: 0.964541	valid_1's auc: 0.968554
[13]	valid_0's auc: 0.96438	valid_1's auc: 0.968572
[14]	valid_0's auc: 0.964383	valid_1's auc: 0.96859
[15]	valid_0's auc: 0.964354	valid_1's auc: 0.96887
[16]	valid_0's auc: 0.964707	valid_1's auc: 0.969376
[17]	valid_0's auc: 0.964613	valid_1's auc: 0.969658
[18]	valid_0's auc: 0.964976	valid_1's auc: 0.969

In [89]:
y_pred3 = clf3.predict_proba(X_test)[:,0]
print(score(y_test, y_pred3)) 

0.40816955684007705


In [90]:
y_pred = 0.4*y_pred1 + 0.3*y_pred2 + 0.3*y_pred3
print(score(y_test, y_pred)) 

0.4034682080924855


In [122]:
y_test[:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)